#### import required packages and initialize the h2o application server

In [1]:
import h2o
import pandas as pd

h2o.init(log_level="INFO")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.20.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.20.04, mixed mode, sharing)
  Starting server from /home/jeff/findChurn/env/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpgz65ury4
  JVM stdout: /tmp/tmpgz65ury4/h2o_jeff_started_from_python.out
  JVM stderr: /tmp/tmpgz65ury4/h2o_jeff_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_jeff_y4t9da
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.943 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


#### load the model and the current customer dataframe

In [2]:
m = h2o.load_model('./churn_model')
current_customers = pd.read_json('Telco_Churn_current_customers.json')
curr = h2o.H2OFrame(current_customers)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


#### Score current customers

In [3]:
# score current current customers for churn using the highest performing model
predict = m.predict(curr)

# bind the prediction to the original dataset and convert to a dataframe
curr = curr.cbind(predict)
current_customers_df = pd.DataFrame(curr.as_data_frame(), columns=curr.names)
all_columns = current_customers_df.columns
for i in all_columns:
    if i[0:12] == "Row Count - ":
        del current_customers_df[i]
print(current_customers_df.loc[:10,["customerID","predict","No","Yes"]])

current_customers.to_json('scored_customers.json')

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
    customerID predict        No       Yes
0   3668-QPYBK     Yes  0.701541  0.298459
1   7795-CFOCW      No  0.959560  0.040440
2   9305-CDSKC     Yes  0.179969  0.820031
3   6713-OKOMC     Yes  0.722420  0.277580
4   9959-WOFKT      No  0.899527  0.100473
5   4190-MFLUW      No  0.800208  0.199792
6   3638-WEABW      No  0.970202  0.029798
7   8665-UTDHZ     Yes  0.492844  0.507156
8   5248-YGIJN      No  0.976534  0.023466
9   6572-ADKRS     Yes  0.764028  0.235972
10  8865-TNMNX      No  0.876855  0.123145


#### shutdown the h2o application server

In [4]:
h2o.cluster().shutdown()

H2O session _sid_9f71 closed.
